# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import print_function
import torch
import json

from data_loader import load_cifar10, get_class_names
from training_utils import train_model, continue_training

# CUDA

In [ ]:
print(f"Is CUDA available? {torch.cuda.is_available()}")
!nvcc --version

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Initialization

In [ ]:
# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader, test_loader, X_train, X_test, Y_train, Y_test = load_cifar10(batch_size=64, seed=42)
class_names = get_class_names()

# Train CNN

In [ ]:
from model import BeterCNN
model = BeterCNN()
model = model.to(device)

print(f"Model device: {next(model.parameters()).device}")

In [ ]:
metrics = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    model_name='cnn',
    num_epochs=300,
    device=device
)

# Continue training

In [ ]:
import json
checkpoint_dir = '/content/cnn_checkpoints'  # Replace with your actual path
with open(f'{checkpoint_dir}/training_metrics.json', 'r') as f:
       metrics = json.load(f)

print(f"Last completed epoch: {metrics['current_epoch']}")

# Continue training
metrics = continue_training(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    model_name='cnn',
    checkpoint_dir=checkpoint_dir,
    target_epochs=700,
    device=device
)

In [ ]:
import shutil
import os

source_dir = '/content/cnn_checkpoints'
destination_dir = '/content/drive/MyDrive/better_cnn_checkpoints'

for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    shutil.copy(source_path, destination_path)
    print(f"Copied: {filename}")

print("All files copied!")